In [1]:
import pandas as pd

routes = pd.read_csv('GTFS/routes.txt')
trips = pd.read_csv('GTFS/trips.txt')
stop_times = pd.read_csv('GTFS/stop_times.txt')
stops = pd.read_csv('GTFS/stops.txt')

In [5]:
def get_stops_for_route(route_id):
    # Шаг 1: Найти все trip_id для данного route_id в trips.txt
    trip_ids = trips[trips['route_id'] == route_id]['trip_id'].unique()
    
    # Шаг 2: Найти все stop_id для этих trip_id в stop_times.txt
    stop_ids = stop_times[stop_times['trip_id'].isin(trip_ids)]['stop_id'].unique()
    
    # Шаг 3: Получить информацию о каждой остановке из stops.txt
    route_stops = stops[stops['stop_id'].isin(stop_ids)]
    
    return route_stops

# Замените 'your_route_id' на реальный идентификатор маршрута из routes.txt
route_id = 208
stops_for_route = get_stops_for_route(route_id)

print(len(stops_for_route))

20


In [9]:
import requests
import plotly.graph_objs as go

# Запрос данных формы (shape) маршрута с сервера
route_id = 'your_route_id'
response = requests.get("http://0.0.0.0:8000/shape/233/")
shapes_data = response.json()

# Извлечение координат из полученных данных
lats = [shape['shape_pt_lat'] for shape in shapes_data if shape['shape_pt_lat'] != None]
lons = [shape['shape_pt_lon'] for shape in shapes_data if shape['shape_pt_lon'] != None]

trace = go.Scattermapbox(
    lat=lats,
    lon=lons,
    mode='lines',
    line=dict(width=2, color='blue'),
)

# Определение параметров карты
layout = go.Layout(
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken='your_mapbox_access_token',  # Укажите ваш ключ доступа к Mapbox
        bearing=0,
        center=dict(lat=sum(lats)/len(lats), lon=sum(lons)/len(lons)),
        pitch=0,
        zoom=10,
        style='light'
    ),
)

# Создание и отображение карты
fig = go.Figure(data=[trace], layout=layout)
fig.show()